In [ ]:
import random
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from numba import njit
import copy
from skopt import Optimizer
import pandas as pd

from numpy.random import Generator, PCG64

from dask.distributed import Client, LocalCluster
import dask.array as da


# This does not change the consistency
# cluster = LocalCluster(n_workers=8)
cluster = LocalCluster()
client = Client(cluster)

#test stock model to compare:
from surprise import SVD,Dataset,Reader


#LOOK: this cell has been developed to test a configuration




# SEED_INT = 56
# random.seed(56)
# np.random.seed(56)
# da.random.RandomState(56)
# da.random.default_rng(56)
# da.random.seed(56)


def funct(block):

    # SEED_INT = 56
    # random.seed(SEED_INT)
    # np.random.seed(SEED_INT)
    # da.random.RandomState(SEED_INT)
    # da.random.default_rng(SEED_INT)
    # da.random.seed(SEED_INT)

    total_sum = 0 

    for row in block:
        seed_int, user_to_data_svd_temp, user_to_data_train_temp, nof_latent_features, epochs, rt, lr = row


        gen = Generator(PCG64(seed_int))


        old_to_new_svd  = dict()
        last_index_svd = 0
        svd_cnt = 0

        for user in user_to_data_svd_temp:
            for row in user: 
                if(row[1] in old_to_new_svd.keys()):
                    row[1] = old_to_new_svd[row[1]]
                else:
                    old_to_new_svd[row[1]] = last_index_svd
                    row[1] = last_index_svd
                    last_index_svd+=1      
                row[0] = svd_cnt
            svd_cnt+=1


        old_to_new_train = copy.deepcopy(old_to_new_svd)
        last_index_train = last_index_svd
        train_cnt = svd_cnt

        for user in user_to_data_train_temp:
            for row in user: 
                if(row[1] in old_to_new_train.keys()):
                    row[1] = old_to_new_train[row[1]]
                else:
                    old_to_new_train[row[1]] = last_index_train
                    row[1] = last_index_train
                    last_index_train+=1      
                row[0] = train_cnt
            train_cnt+=1


        target_rating_train = []
        train_list = []


        movies_order_svd = set()
        overall_average_svd = 0 
        cnt_svd = 0


        for user in user_to_data_svd_temp:
            for movie in user:
                movies_order_svd.add(movie[1])
                train_list.append([int(movie[0]), int(movie[1]), float(movie[2])])
                overall_average_svd+=float(movie[2])
                cnt_svd += 1



        movies_order_train = copy.deepcopy(movies_order_svd)
        overall_average_train = overall_average_svd 
        cnt_train = cnt_svd
        train_rating_to_predict = []

        for user in user_to_data_train_temp:
            # rand_num  = random.randint(0, len(user)-1)
            rand_num  = gen.integers(0, len(user))
            index = 0
            for movie in user:
                movies_order_train.add(movie[1])
                if(index == rand_num):
                    train_rating_to_predict.append([int(movie[0]), int(movie[1])])
                    target_rating_train.append(float(movie[2]))
                else:
                    overall_average_train+=float(movie[2])
                    cnt_train += 1
                    train_list.append([int(movie[0]), int(movie[1]), float(movie[2])])
                index+=1


        overall_average_train = overall_average_train/cnt_train

        # random.shuffle(train_list)
        gen.shuffle(train_list)


        @njit
        def epoch(list, b1, b2, p, q, overall_average, lr, rt):
            for row in list:
                u = int(row[0])
                i = int(row[1])
                r = row[2]

                pred = overall_average+b1[u]+b2[i]+np.dot(p[u],q[i])
                error = r-pred
                b1[u] += lr*(error- rt*b1[u])
                b2[i] += lr*(error- rt*b2[i])
                temp = lr*(error*q[i] -rt*p[u])
                q[i] += lr*(error*p[u] -rt*q[i])
                p[u] += temp





        def svd_iterative(list, n, epochs, rt, lr, overall_average, nof_users, nof_movies):

            # q = np.random.normal(0, .1, (nof_movies, n))
            # p = np.random.normal(0, .1, (nof_users, n))

            q = gen.normal(0, .1, (nof_movies, n))
            p = gen.normal(0, .1, (nof_users, n))


            b1 = np.zeros(nof_users)
            b2 = np.zeros(nof_movies)

            np_array = np.array(list)

            for _ in range(epochs):
                epoch(np_array, b1, b2, p, q, overall_average, lr, rt)

            return b1, b2, p, q


        b1, b2, p, q = svd_iterative(train_list, nof_latent_features, epochs, rt, lr,
                                    overall_average_train, len(user_to_data_svd_temp)+len(user_to_data_train_temp), len(movies_order_train))

        feature_3_train = [overall_average_train + b1[pair[0]]+b2[pair[1]]
                                    +np.dot(p[pair[0]],q[pair[1]]) for pair in train_rating_to_predict]
    
        total_sum+=mean_squared_error(target_rating_train, feature_3_train, squared = False)

    return (np.array([[total_sum]], dtype="float32"))





mse_sum =0 
runs = 80



nof_svd_users, nof_train_users, nof_latent_features, epochs, rt, lr = (355, 123, 196, 204, 0.03407177995884917, 0.03668743198104899)


#LOOK: this makes this cell slightly inconsistent with the best call in the cell below
user_to_data_svd_copy = copy.deepcopy(user_to_data_svd)
user_to_data_train_copy = copy.deepcopy(user_to_data_train)


# random.shuffle(user_to_data_svd_copy)
# random.shuffle(user_to_data_train_copy)


# user_to_data_svd_list = [user_to_data_svd_copy[i*nof_svd_users : (i+1)*nof_svd_users] for i in range(runs)]
# user_to_data_train_list = [user_to_data_train_copy[i*nof_train_users : (i+1)*nof_train_users] for i in range(runs)]




outer = Generator(PCG64(15))


parameters_list = []




for _ in range(runs):
    parameters_list.append([outer.integers(0,100),
                            copy.deepcopy(outer.choice(user_to_data_svd_copy, nof_svd_users, replace = False)),
                            copy.deepcopy(outer.choice(user_to_data_train_copy, nof_train_users, replace = False)),
                            nof_latent_features, epochs, rt, lr])




test = np.array(parameters_list, dtype="object")
dask_array = da.from_array(test, chunks=(10,7))
results = dask_array.map_blocks(funct, chunks = (1,1), dtype="float32").compute()


for row in results:
    mse_sum+= row[0]


print("fixed parameters rmse score:",mse_sum/runs)


client.close()
cluster.close()


# seed_int = 5
# fixed parameters rmse score: 1.0399177312850951
# seed_int = 10
# fixed parameters rmse score: 1.0447407484054565
# seed_int = 15
# fixed parameters rmse score: 1.0471360564231873